In [110]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
import findspark
findspark.init()

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Pedidos") \
    .config('spark.ui.port', '4050') \
    .getOrCreate()

In [111]:
spark

Dados do pedido

In [112]:
path = './arquivos/olist_orders_dataset.csv'
df_pedidos = spark.read.csv(path, sep=',', inferSchema=True, header=True)

In [113]:
df_pedidos = df_pedidos\
    .select('order_id', 'customer_id', 'order_purchase_timestamp')\
    .filter(df_pedidos['order_status'].like('%delivered%'))

In [114]:
df_pedidos.limit(5).toPandas()

/usr/local/lib/python3.10/dist-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,order_id,customer_id,order_purchase_timestamp
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,2017-10-02 10:56:33
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,2018-07-24 20:41:37
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,2018-08-08 08:38:49
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,2017-11-18 19:28:06
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,2018-02-13 21:18:39


dados do cliente


In [115]:
path = './arquivos/olist_customers_dataset.csv'
df_clientes = spark.read.csv(path, sep=',', inferSchema=True, header=True)

In [116]:
df_clientes = df_clientes\
    .select('customer_id', 'customer_city', 'customer_state')

In [117]:
df_clientes.limit(5).toPandas()

,customer_id,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,campinas,SP


In [118]:
df_pedidos = df_pedidos.join(df_clientes, 'customer_id', how='inner')\
    .sort('customer_id')

dados de pagamentos

In [119]:
path = './arquivos/olist_order_payments_dataset.csv'
df_pagamentos = spark.read.csv(path, sep=',', inferSchema=True, header=True)

In [120]:
df_pagamentos.limit(5).toPandas()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


In [121]:
df_pedidos = df_pedidos.join(df_pagamentos, 'order_id', how='inner')\
    .sort('order_id')

dados dos itens do pedido

In [122]:
path = './arquivos/olist_order_items_dataset.csv'
df_pedidos_itens = spark.read.csv(path, sep=',', inferSchema=True, header=True)

In [123]:
df_pedidos_itens.limit(5).toPandas()
df_pedidos = df_pedidos.join(df_pedidos_itens, 'order_id', how='inner')\
    .sort('order_id')

/usr/local/lib/python3.10/dist-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


dados do produtos 

In [124]:
path = './arquivos/olist_products_dataset.csv'
df_produtos = spark.read.csv(path, sep=',', inferSchema=True, header=True)

In [125]:

df_produtos = df_produtos\
    .select('product_id', 'product_category_name')
df_produtos.limit(5).toPandas()

,product_id,product_category_name
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria
1,3aa071139cb16b67ca9e5dea641aaa2f,artes
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer
3,cef67bcfe19066a932b7673e239eb23d,bebes
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas


In [126]:
df_pedidos = df_pedidos.join(df_produtos, 'product_id', how='inner')
df_pedidos.limit(5).toPandas()

/usr/local/lib/python3.10/dist-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)
/usr/local/lib/python3.10/dist-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,product_id,order_id,customer_id,order_purchase_timestamp,customer_city,customer_state,payment_sequential,payment_type,payment_installments,payment_value,order_item_id,seller_id,shipping_limit_date,price,freight_value,product_category_name
0,08279c494018541f71443c07d77560f8,0032d07457ae9c806c79368d7d9ce96b,d356c20816dc75a309628b5c17d89468,2018-03-10 18:53:06,rio de janeiro,RJ,1,credit_card,2,186.19,1,e333046ce6517bd8bb510291d44f0130,2018-03-15 19:28:51,159.00,27.19,utilidades_domesticas
1,9545d45c37449ccbc376de3a04c66e71,0045e3085f083f0f38d24bb3f22e6593,eb729faa993caad20442243265804a7c,2017-08-10 16:13:36,sao jose dos campos,SP,1,credit_card,3,130.74,1,431af27f296bc6519d890aa5a05fdb11,2017-08-17 07:15:16,116.90,13.84,beleza_saude
2,19421075ae0b585f2dc13ff149e2119d,0079bca8e89bd52fdb87408e4f3fb94d,137f50d5bfbcfb9578c50605cc9adb16,2018-05-03 18:55:20,sao paulo,SP,1,boleto,1,57.29,1,4c2b230173bb36f9b240f2b8ac11786e,2018-05-10 03:31:09,49.90,7.39,esporte_lazer
3,e819fddd6622f30e872bfc89471c1408,007ff0b0f79be782f633fc3b18cbe337,fc021f2dd664b49455528422c77b6aab,2018-02-03 17:46:53,belo horizonte,MG,1,credit_card,1,55.11,1,7139dc5186aa238b006e066d8fbab2e4,2018-02-08 20:10:10,39.00,16.11,moveis_decoracao
4,d64e758afad411049a45e42c9a259241,0094bd07f49fed90209ffa62d1ef26d6,de0c1a4d8c367c58d66e61dfa379f4cf,2018-05-01 02:04:16,praia grande,SP,1,boleto,1,18.92,1,5670f4db5b62c43d542e1b2d56b0cf7c,2018-05-09 04:12:53,11.53,7.39,perfumaria


dados vendedor 

In [127]:
path = './arquivos/olist_sellers_dataset.csv'
df_vendedor  = spark.read.csv(path, sep=',', inferSchema=True, header=True)

In [128]:
df_vendedor = df_vendedor\
    .select('seller_id', 'seller_city',  'seller_state')

In [129]:
df_pedidos = df_pedidos.join(df_vendedor, 'seller_id', how='inner')
df_pedidos.limit(5).toPandas()

/usr/local/lib/python3.10/dist-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)
/usr/local/lib/python3.10/dist-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,seller_id,product_id,order_id,customer_id,order_purchase_timestamp,customer_city,customer_state,payment_sequential,payment_type,payment_installments,payment_value,order_item_id,shipping_limit_date,price,freight_value,product_category_name,seller_city,seller_state
0,e333046ce6517bd8bb510291d44f0130,08279c494018541f71443c07d77560f8,0032d07457ae9c806c79368d7d9ce96b,d356c20816dc75a309628b5c17d89468,2018-03-10 18:53:06,rio de janeiro,RJ,1,credit_card,2,186.19,1,2018-03-15 19:28:51,159.00,27.19,utilidades_domesticas,ipaussu,SP
1,431af27f296bc6519d890aa5a05fdb11,9545d45c37449ccbc376de3a04c66e71,0045e3085f083f0f38d24bb3f22e6593,eb729faa993caad20442243265804a7c,2017-08-10 16:13:36,sao jose dos campos,SP,1,credit_card,3,130.74,1,2017-08-17 07:15:16,116.90,13.84,beleza_saude,ribeirao preto,SP
2,4c2b230173bb36f9b240f2b8ac11786e,19421075ae0b585f2dc13ff149e2119d,0079bca8e89bd52fdb87408e4f3fb94d,137f50d5bfbcfb9578c50605cc9adb16,2018-05-03 18:55:20,sao paulo,SP,1,boleto,1,57.29,1,2018-05-10 03:31:09,49.90,7.39,esporte_lazer,sao paulo,SP
3,7139dc5186aa238b006e066d8fbab2e4,e819fddd6622f30e872bfc89471c1408,007ff0b0f79be782f633fc3b18cbe337,fc021f2dd664b49455528422c77b6aab,2018-02-03 17:46:53,belo horizonte,MG,1,credit_card,1,55.11,1,2018-02-08 20:10:10,39.00,16.11,moveis_decoracao,amparo,SP
4,5670f4db5b62c43d542e1b2d56b0cf7c,d64e758afad411049a45e42c9a259241,0094bd07f49fed90209ffa62d1ef26d6,de0c1a4d8c367c58d66e61dfa379f4cf,2018-05-01 02:04:16,praia grande,SP,1,boleto,1,18.92,1,2018-05-09 04:12:53,11.53,7.39,perfumaria,sao paulo,SP


In [138]:
df_pedidos.select([f.count(f.when(f.isnull(c), 1)).alias(c) for c in df_pedidos.columns]).show()

+---------+----------+--------+-----------+------------------------+-------------+--------------+------------------+------------+--------------------+-------------+-------------+-------------------+-----+-------------+---------------------+-----------+------------+
|seller_id|product_id|order_id|customer_id|order_purchase_timestamp|customer_city|customer_state|payment_sequential|payment_type|payment_installments|payment_value|order_item_id|shipping_limit_date|price|freight_value|product_category_name|seller_city|seller_state|
+---------+----------+--------+-----------+------------------------+-------------+--------------+------------------+------------+--------------------+-------------+-------------+-------------------+-----+-------------+---------------------+-----------+------------+
|        0|         0|       0|          0|                       0|            0|             0|                 0|           0|                   0|            0|            0|                  0|    

In [131]:
df_pedidos.count()

115035

In [137]:
df_pedidos.select([f.when(f.isnull('product_category_name'), 1).alias('product_category_name2'), '*'] )\
    .where("product_category_name2==1").show()

+----------------------+---------+----------+--------+-----------+------------------------+-------------+--------------+------------------+------------+--------------------+-------------+-------------+-------------------+-----+-------------+---------------------+-----------+------------+
|product_category_name2|seller_id|product_id|order_id|customer_id|order_purchase_timestamp|customer_city|customer_state|payment_sequential|payment_type|payment_installments|payment_value|order_item_id|shipping_limit_date|price|freight_value|product_category_name|seller_city|seller_state|
+----------------------+---------+----------+--------+-----------+------------------------+-------------+--------------+------------------+------------+--------------------+-------------+-------------+-------------------+-----+-------------+---------------------+-----------+------------+
+----------------------+---------+----------+--------+-----------+------------------------+-------------+--------------+-------------

In [136]:
df_pedidos = df_pedidos.na.drop()

top 10 por estados 

In [148]:
df_pedidos\
    .select('customer_state')\
    .groupBy('customer_state')\
    .count()\
    .orderBy('count', ascending = 0)\
    .show(10, truncate=False) 

+--------------+-----+
|customer_state|count|
+--------------+-----+
|SP            |47840|
|RJ            |14651|
|MG            |13236|
|RS            |6288 |
|PR            |5790 |
|SC            |4160 |
|BA            |3859 |
|DF            |2389 |
|GO            |2319 |
|ES            |2288 |
+--------------+-----+
only showing top 10 rows



media por estados 

In [157]:
df_pedidos\
    .select('order_id', 'customer_state')\
    .groupBy('customer_state')\
    .agg(
        f.count("order_id").alias("total_state"),
        f.avg("order_id").alias("media")
        
    )\
    .orderBy('total_state', ascending = 0)\
    .show(10, truncate=False) 

+--------------+-----+-----+
|customer_state|id   |media|
+--------------+-----+-----+
|SP            |47840|null |
|RJ            |14651|null |
|MG            |13236|null |
|RS            |6288 |null |
|PR            |5790 |null |
|SC            |4160 |null |
|BA            |3859 |null |
|DF            |2389 |null |
|GO            |2319 |null |
|ES            |2288 |null |
+--------------+-----+-----+
only showing top 10 rows

